#### Parse additional historical features from the EHR (medications, outpatient visits) and time-series data (blood tests, vital signs)

In [1]:
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.patches import Patch
from matplotlib.dates import DateFormatter
from datetime import timedelta, datetime
from tqdm import tqdm
import numpy as np
from scipy import stats, special
from tableone import TableOne

import os
import json
import re
import pprint
import missingno as msno
from statannotations.Annotator import Annotator
import warnings

pd.set_option('display.max_rows', None)

#### Load linked demographics data and helper-functions

In [ ]:
demo_data = pd.read_csv('../outputs/linked_data/linked_demographics.csv')
print(demo_data.shape, demo_data.subject_id.nunique())

In [2]:
core_path = '../../data/MIMIC-IV/mimiciv/3.1'
ed_path = '../../data/MIMIC-IV/mimic-iv-ed/3.1'
demo_path = '../outputs/linked_data/linked_demographics.csv'
ndc_path = '../../data/MIMIC-IV/config/NDC_product_table.csv'

In [ ]:
meds_data = read_prescriptions_table(core_path, demo_path)

In [ ]:
meds_feat = prepare_prescription_features(meds_data, demo_data)

In [ ]:
meds_feat.isnull().sum()

In [ ]:
meds_feat.total_n_presc.describe()

In [ ]:
for col in meds_feat.columns.tolist():
    if 'dsl' in col:
        print(meds_feat[col].value_counts().head(2))

In [52]:
meds_feat.to_csv('../outputs/linked_data/linked_meds_demographics.csv', index=False)

In [16]:
meds_feat = pd.read_csv('../outputs/linked_data/linked_meds_demographics.csv')

In [6]:
poe_test = dataframe_from_csv(os.path.join(core_path, 'hosp/poe.csv.gz'))

#### Specialty-related orders

In [ ]:
meds_feat = prepare_admin_features(poe_test, meds_feat)

In [25]:
meds_feat.to_csv('../outputs/linked_data/linked_meds_proc_demographics.csv', index=False)

In [ ]:
meds_feat.isnull().sum()

In [ ]:
meds_feat.shape

In [ ]:
for col in meds_feat.columns.tolist():
    if 'admin' in col:
        print(meds_feat[col].value_counts().head())
        print(meds_feat[col].describe())

#### Get vital signs and lab tests

In [ ]:
measure_data = prepare_vitals(measures_test, vitals_test, meds_feat, '../outputs/linked_data/measures_ts.csv')

In [ ]:
print(measure_data.shape, measure_data.subject_id.nunique())
print(measure_data[measure_data.label.str.contains('blood pressure')].shape,
    measure_data[measure_data.label.str.contains('blood pressure')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('BMI')].shape,
    measure_data[measure_data.label.str.contains('BMI')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('Temperature')].shape,
    measure_data[measure_data.label.str.contains('Temperature')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('Respiratory rate')].shape,
    measure_data[measure_data.label.str.contains('Respiratory rate')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('Oxygen saturation')].shape,
    measure_data[measure_data.label.str.contains('Oxygen saturation')].subject_id.nunique())

In [ ]:
measure_data.head()

In [40]:
measures_test = dataframe_from_csv(os.path.join(core_path, 'hosp/omr.csv.gz'))

In [ ]:
measures_test.isnull().sum()

In [ ]:
measures_test.head()

In [36]:
vitals_test = dataframe_from_csv(os.path.join(ed_path, 'ed/vitalsign.csv.gz'))

In [ ]:
vitals_test.isnull().sum()

In [ ]:
vitals_test.result_name.value_counts()

##### Blood tests

In [4]:
demo_data = pd.read_csv('../outputs/linked_data/linked_demographics.csv')

In [18]:
#### Get most common lab tests as reference data
labs_lkup = pd.read_csv(os.path.join(core_path, 'hosp/labevents.csv'),
                        chunksize=10000000, low_memory=False)
labs_names = pd.read_csv(os.path.join(core_path, 'hosp/d_labitems.csv.gz'), compression='gzip')
labs_names = labs_names[['itemid', 'label']]
labs_lkup = pd.merge(labs_lkup.get_chunk(10000000), labs_names, on='itemid', how='left')
labs_lkup = labs_lkup.dropna(subset=['itemid', 'label', 'valuenum', 'valueuom'])
labs_lkup['label'] = labs_lkup['label'].str.lower().str.strip().str.replace(' ', '_').str.replace(',', '').str.replace('"', '')
labs_lkup = labs_lkup[['subject_id', 'itemid', 'label']].groupby(['itemid', 'label']).size().reset_index(name='n_tests')
labs_lkup = labs_lkup.sort_values(['n_tests'], ascending=False).head(50)
labs_lkup.to_csv('../outputs/linked_data/labs_lkup.csv', index=False)

In [ ]:
labs_lkup = pd.read_csv(os.path.join(core_path, 'hosp/labevents.csv'),
                        chunksize=10000, low_memory=False)
labs_lkup.get_chunk(10).charttime

In [19]:
### Export lab test ids as txt
lab_items_new = labs_lkup.itemid.tolist()
with open('../outputs/linked_data/lab_items.txt', 'w') as f:
    for item in lab_items_new:
        f.write("%s\n" % item)

In [ ]:
labs_data = pl.scan_csv(os.path.join(core_path, 'hosp/labevents.csv'), try_parse_dates=True)
d_items = (pl.read_csv(os.path.join(core_path, "hosp/d_labitems.csv.gz")).lazy().select(["itemid", "label"]))
# merge labitem id's with dict
labs_data = labs_data.join(d_items, how='left', on="itemid")
# select relevant columns
labs_data = (labs_data.select(["subject_id", "charttime", "itemid", "label", "value", "valueuom"])
        .with_columns(charttime=pl.col("charttime").cast(pl.Datetime), linksto=pl.lit("labevents")))
# get eligible lab tests prior to current episode
labs_data = labs_data.join(pl.from_pandas(demo_data[['subject_id', 'edregtime']]).lazy().
                           with_columns(edregtime=pl.col("edregtime").str.to_datetime(format="%Y-%m-%d %H:%M:%S")), 
                           how='left', on="subject_id")
labs_data = labs_data.filter(pl.col("charttime") <= pl.col("edregtime")).drop(["edregtime"])
# get most common items
labs_data = labs_data.filter(pl.col("itemid").is_in(set(lab_items_new)))
labs_data = labs_data.with_columns(
    pl.col("label").str.to_lowercase().str.replace(" ", "_").str.replace(",", "").str.replace('"', "").str.replace(" ", "_"),
    pl.col("charttime").str.replace("T", " ").str.strip_chars()
)
lab_events = labs_data.with_columns(
        value=pl.when(pl.col("value") == ".").then(None).otherwise(pl.col("value"))
)
lab_events = lab_events.with_columns(
    value=pl.when(pl.col("value").str.contains("_|<|ERROR"))
    .then(None)
    .otherwise(pl.col("value"))
    .cast(pl.Float64, strict=False)  # Attempt to cast to Float64, set invalid values to None
)
labs_data = labs_data.drop_nulls()

# Remove outliers using 2 std from mean
lab_events = lab_events.with_columns(mean=pl.col("value").mean().over(pl.count("label")))
lab_events = lab_events.with_columns(std=pl.col("value").std().over(pl.count("label")))
lab_events = lab_events.filter(
    (pl.col("value") < pl.col("mean") + pl.col("std") * 2)
    & (pl.col("value") > pl.col("mean") - pl.col("std") * 2)
).drop(["mean", "std"])

lab_events = lab_events.collect()
lab_events.write_csv(include_header=True, file='../outputs/linked_data/labs_ts.csv')

In [ ]:
labs_test = pd.read_csv('../outputs/linked_data/labs_ts.csv', chunksize=10)
labs_test.get_chunk(10).head()

In [ ]:
lab_events.head(10)

In [ ]:
print(labs_data.columns)

In [ ]:
labs_data.limit(10000).collect().head(10)